# 안쓰는 함수

In [ ]:
# feature shift
# N(lag)행 만큼 데이터를 밀어냄
def create_lag_feats(data, lags, cols):
    
    lag_cols = []
    temp = data.copy()
    for col in cols:
        for lag in lags:
            temp[col + '_lag_%s'%lag] = temp[col].shift(lag)
            temp['Target1'] = temp['TARGET']
            # fill values forward: 각 행의 앞단의 값을 채워 넣는 것
            temp['Target2'] = temp['TARGET'].shift(-48).fillna(method='ffill')  
            lag_cols.append(col + '_lag_%s'%lag)

    return temp, lag_cols

def preprocess_data(data, target_lags=[48], weather_lags=[48], is_train=True):
    
    temp = data.copy()

    if is_train==True:          
    
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
        
        # Minute 추가
        return temp[['Hour', 'Minute'] + temp_lag_cols1 + temp_lag_cols2 + ['Target1', 'Target2']].dropna()

    elif is_train==False:    
        
        temp, temp_lag_cols1 = create_lag_feats(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
                              
        return temp[['Hour', 'Minute'] + temp_lag_cols1 + temp_lag_cols2].dropna()
    
# 하루전날 데이터를 통해서 새로운 셋 구축
# 허나... 지금 현재 방법론을 봤을 때 Day6의 테스트 데이터로 추론을 해야하는데
# Day5로 추론하는 문제가 있는데 오히려 성능이 감소됨;;;
df_train = preprocess_data(train, target_lags=[48], weather_lags=[48], is_train=True)

# test셋 구축
df_test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path).iloc[-48:]
#     temp = preprocess_data(temp, target_lags=[48], weather_lags=[48], is_train=False).iloc[-48:]
    df_test.append(temp)

X_test = pd.concat(df_test)

X_test.drop(columns = ['Day'], inplace = True)
X_test = X_test[['Hour','Minute','TARGET','DHI','DNI','WS','RH','T']]
X_test.columns = df_train.columns.tolist()[:-2]

X_test['DHI_lag_48'] = X_test['DHI_lag_48'].astype('float64')
X_test['DNI_lag_48'] = X_test['DNI_lag_48'].astype('float64')

X_test.shape


# feature shift
# N(lag)행 만큼 데이터를 밀어냄
def create_lag_feats_Day23(data, lags, cols):
    
    lag_cols = []
    temp = data.copy()
    for col in cols:
        for lag in lags:
            temp[col + '_lag_%s'%lag] = temp[col].shift(lag)
            # 이틀 뒤의 타겟값
            temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')
            # fill values forward: 각 행의 앞단의 값을 채워 넣는 것
            # 3일 뒤의 타겟값
            temp['Target2'] = temp['TARGET'].shift(-96).fillna(method='ffill')  
            lag_cols.append(col + '_lag_%s'%lag)

    return temp, lag_cols

def preprocess_data_Day23(data, target_lags=[48], weather_lags=[48], is_train=True):
    
    temp = data.copy()

    if is_train==True:          
    
        temp, temp_lag_cols1 = create_lag_feats_Day23(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats_Day23(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
        
        # Minute 추가
        return temp[['Hour', 'Minute'] + temp_lag_cols1 + temp_lag_cols2 + ['Target1', 'Target2']].dropna()

    elif is_train==False:    
        
        temp, temp_lag_cols1 = create_lag_feats_Day23(temp, target_lags, ['TARGET'])
        temp, temp_lag_cols2 = create_lag_feats_Day23(temp, weather_lags, ['DHI', 'DNI', 'WS', 'RH', 'T'])
                              
        return temp[['Hour', 'Minute'] + temp_lag_cols1 + temp_lag_cols2].dropna()
    
# 하루전날 데이터를 통해서 새로운 셋 구축
# 허나... 지금 현재 방법론을 봤을 때 Day6의 테스트 데이터로 추론을 해야하는데
# Day5로 추론하는 문제가 있는데 오히려 성능이 감소됨;;;
df_train_day23 = preprocess_data_Day23(train, target_lags=[48], weather_lags=[48], is_train=True)

# test셋 구축
df_test = []

for i in range(81):
    file_path = './data/test/' + str(i) + '.csv'
    # Day5 데이터를 추출
    temp = pd.read_csv(file_path).iloc[-96:-48]
#     temp = preprocess_data(temp, target_lags=[48], weather_lags=[48], is_train=False).iloc[-48:]
    df_test.append(temp)

X_test_day23 = pd.concat(df_test)

X_test_day23.drop(columns = ['Day'], inplace = True)
X_test_day23 = X_test_day23[['Hour','Minute','TARGET','DHI','DNI','WS','RH','T']]
X_test_day23.columns = df_train_day23.columns.tolist()[:-2]

X_test_day23['DHI_lag_48'] = X_test_day23['DHI_lag_48'].astype('float64')
X_test_day23['DNI_lag_48'] = X_test_day23['DNI_lag_48'].astype('float64')

X_test_day23.shape